In [1]:
from fmr_scripts import FMR #if running in another directory - requires PYTHONPATH or sys.path to contain directory containing fmr_scripts
#import FMR #if running in same directory
import numpy as np
import matplotlib.pyplot as plt
import glob,os
from scipy.interpolate import interp1d

In [ ]:
filenames = glob.glob('*.npz')
filename = filenames[np.argwhere(np.array([f.find('-fitparams') for f in filenames]) > -1)[0][0]]

In [ ]:
extractedparams = np.load(filename)
extractedparams.files

In [ ]:
NPeaks = [len(f) for f in extractedparams['resonantFieldsOe']]
print NPeaks

In [ ]:
#essentially a reslicing
def extract(data, field, n, whichcurves):
    return np.array([data[field][i][n] for i in whichcurves])

N = np.max(NPeaks)
freqsN = []
angsN = []
HFMRN = []
errHFMRN = []
deltaHN = []
errdeltaHN = []
phaseLorentzianN = []
errphaseLorentzianN = []
peakMagnitudeN = []
errpeakMagnitudeN = []
for n in np.arange(0,N):
    whichcurves = np.where(np.array(NPeaks) > n)[0]
    freqsN.append(extractedparams['frequenciesGHz'][whichcurves])
    angsN.append(extractedparams['anglesdeg'][whichcurves])
    HFMRN.append(extract(extractedparams,'resonantFieldsOe',n,whichcurves))
    errHFMRN.append(extract(extractedparams,'resonantFieldsErrorOe',n,whichcurves))
    deltaHN.append(extract(extractedparams,'linewidthsOe',n,whichcurves))
    errdeltaHN.append(extract(extractedparams,'linewidthsErrorOe',n,whichcurves))
    phaseLorentzianN.append(extract(extractedparams,'peakPhasesdeg',n,whichcurves))
    errphaseLorentzianN.append(extract(extractedparams,'peakPhasesErrordeg',n,whichcurves))
    peakMagnitudeN.append(extract(extractedparams,'peakMagnitude',n,whichcurves))
    errpeakMagnitudeN.append(extract(extractedparams,'peakMagnitudeError',n,whichcurves))

#### Analysis picking a single mode

#### Systematically determine which frequency points to discard (saturation magnetization? poor waveguide coupling?)

In [ ]:
#need to take into account saturation magnetization
h = 6.62607004e-34*1e7 #erg-s
muB = 9.274000999e-21 #erg/G
#mu0 = 1.00000037 #G/Oe - https://www.engineeringtoolbox.com/permeability-d_1923.html
mu0 = 1. #actually measured in G not Oe
giga = 1e9
OetoAperm = 1000/(4*np.pi)

In [ ]:
Meff = []
gop = []
HFMRfit = []

deltaHfit = []
deltaH0 = []
alpha = []

start = np.arange(0,len(freqsN[0])-1)
for i in start:
    freqslice = slice(i,None)
    HFMRfit.append(np.polyfit(HFMRN[0][freqslice],freqsN[0][freqslice],1))
    Meff.append(-HFMRfit[i][1]/(HFMRfit[i][0]*4*np.pi))
    gop.append(HFMRfit[i][0]*h/(muB*mu0)*giga)
    
    deltaHfit.append(np.polyfit(freqsN[0][freqslice],deltaHN[0][freqslice],1,w=1./errdeltaHN[0][freqslice]))
    deltaH0.append(deltaHfit[i][1])
    alpha.append(deltaHfit[i][0]*HFMRfit[i][0])

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],HFMRN[0],fmt='.',yerr=errHFMRN[0])
for i in start:
    #if i < 12:
    plt.plot(np.poly1d(HFMRfit[i])(HFMRN[0]),HFMRN[0],'-',label=str(i))
plt.legend()
plt.xlabel('f (GHz)')
plt.ylabel('HFMR OOP (Oe)')

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],deltaHN[0],fmt='.',yerr=errdeltaHN[0])
for i in start:
    #if i < 6:
    plt.plot(freqsN[0],np.poly1d(deltaHfit[i])(freqsN[0]),'-',label=str(i))
plt.legend()
plt.xlabel('f (GHz)')
plt.ylabel('deltaH OOP (Oe)')

In [ ]:
%matplotlib notebook
plt.plot(start,Meff,'.')
plt.xlabel('starting index of fit')
plt.ylabel('Meff')

In [ ]:
%matplotlib notebook
plt.plot(start,gop,'.')
plt.xlabel('starting index of fit')
plt.ylabel('gop')

In [ ]:
%matplotlib notebook
plt.plot(start,deltaH0,'.')
plt.xlabel('starting index of fit')
plt.ylabel('deltaH0')

In [ ]:
%matplotlib notebook
plt.plot(start,alpha,'.')
plt.xlabel('starting index of fit')
plt.ylabel('alpha')

In [ ]:
start = 2 #defined visually from above

Meff = []
gop = []
HFMRfit = []

deltaHfit = []
deltaH0 = []
alpha = []

stop = np.arange(start+2,len(freqsN[0]))
for i in stop:
    freqslice = slice(start,i)
    HFMRfit.append(np.polyfit(HFMRN[0][freqslice],freqsN[0][freqslice],1))
    Meff.append(-HFMRfit[i-(start+2)][1]/(HFMRfit[i-(start+2)][0]*4*np.pi))
    gop.append(HFMRfit[i-(start+2)][0]*h/(muB*mu0)*giga)
    
    deltaHfit.append(np.polyfit(freqsN[0][freqslice],deltaHN[0][freqslice],1,w=1./errdeltaHN[0][freqslice]))
    deltaH0.append(deltaHfit[i-(start+2)][1])
    alpha.append(deltaHfit[i-(start+2)][0]*HFMRfit[i-(start+2)][0])

In [ ]:
%matplotlib notebook
plt.errorbar(freqsN[0],deltaHN[0],fmt='.',yerr=errdeltaHN[0])
for i in stop:
    #if i > 5:
    plt.plot(freqsN[0],np.poly1d(deltaHfit[i-4])(freqsN[0]),'-',label=str(i))
plt.legend()
plt.xlabel('f (GHz)')
plt.ylabel('deltaH OOP (Oe)')

In [ ]:
%matplotlib notebook
plt.plot(stop,alpha,'.')
plt.xlabel('stopping index of fit')
plt.ylabel('alpha')

In [ ]:
stop = len(freqsN[0]) #visually defined
index = stop-start-2-1

#### Display data

In [ ]:
print 'Meff: '+str(Meff[index])+' emu/cm^3'
print 'gop: '+str(gop[index])

%matplotlib notebook
plt.errorbar(freqsN[0],HFMRN[0],fmt='.',yerr=errHFMRN[0])
if start != 0 or stop != len(freqsN[0]):
    freqslice = slice(start,stop)
    plt.errorbar(freqsN[0][freqslice],HFMRN[0][freqslice],fmt='.',yerr=errHFMRN[0][freqslice])
plt.plot(np.poly1d(HFMRfit[index])(HFMRN[0]),HFMRN[0],'-',label='Kittel eq.')
plt.legend()
plt.ylabel(r'H$_{FMR}$ OOP (Oe)')
plt.xlabel('Frequency (GHz)')

In [ ]:
print "deltaH_0: "+str(deltaH0[index])+' Oe'
print "alpha: "+str(alpha[index])

%matplotlib notebook
plt.errorbar(freqsN[0],deltaHN[0],fmt='.',yerr=errdeltaHN[0])
if start != 0 or stop != len(freqsN[0]):
    plt.errorbar(freqsN[0][freqslice],deltaHN[0][freqslice],fmt='.',yerr=errdeltaHN[0][freqslice])
plt.plot(freqsN[0],np.poly1d(deltaHfit[index])(freqsN[0]),'-')
plt.ylabel('FMR Linewidth OOP (Oe)')
plt.xlabel('Frequency (GHz)')

In [ ]:
fname = filename.partition('-')[0]
outfile = fname+'-params'
np.savez(outfile, frequenciesGHz = freqsN[0][freqslice],
         resonantFieldOe = HFMRN[0][freqslice], linewidthOe = deltaHN[0][freqslice],
         resonantFieldErrorOe = errHFMRN[0][freqslice], linewidthErrorOe = errdeltaHN[0][freqslice],
         peakPhasedeg = phaseLorentzianN[0][freqslice], peakMagnitude = peakMagnitudeN[0][freqslice],
         MeffectiveOe = Meff[index], gilbertDamping = alpha[index],
         orbitalgop = gop[index], intrinsiclinewidthOe = deltaH0[index],
         resonantFieldFitCoefficients = HFMRfit[index], linewidthFitCoefficients = deltaHfit[index])
#insert CRO thickness, LSMO thickness into this file??